## 第2章 因果関係

In [1]:
ENV["LINES"] = 10
ENV["COLUMNS"] = 1000

1000

In [71]:
using DataFrames, DataFramesMeta, CSV, CategoricalArrays, FreqTables, Statistics, StatsBase

## 2.1 労働市場における人種差別

In [20]:
resume_df = CSV.read("../../data/CAUSALITY/resume.csv", DataFrame, missingstring=["NA"])

,firstname,sex,race,call
,String15,String7,String7,Int64
1,Allison,female,white,0
2,Kristen,female,white,0
3,Lakisha,female,black,0
4,Latonya,female,black,0
5,Carrie,female,white,0
6,Jay,male,white,0
7,Jill,female,white,0
8,Kenya,female,black,0
9,Latonya,female,black,0


In [21]:
describe(resume_df)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,firstname,,Aisha,,Tyrone,0,String15
2,sex,,female,,male,0,String7
3,race,,black,,white,0,String7
4,call,0.0804928,0,0.0,1,0,Int64


In [24]:
resume_df = @transform resume_df begin
    :firstname = categorical(:firstname)
    :sex = categorical(:sex, compress=true)
    :race = categorical(:race, compress=true)
end
describe(resume_df)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Nothing,Cat…,Nothing,Cat…,Int64,DataType
1,race,,black,,white,0,"CategoricalValue{String7, UInt8}"


In [26]:
freqtable(resume_df, :race, :call)

2×2 Named Matrix{Int64}
race ╲ call │    0     1
────────────┼───────────
"black"     │ 2278   157
"white"     │ 2200   235

In [32]:
prop(freqtable(resume_df, :race, :call), margins=1)

2×2 Named Matrix{Float64}
race ╲ call │         0          1
────────────┼─────────────────────
"black"     │  0.935524  0.0644764
"white"     │  0.903491  0.0965092

In [36]:
@combine(groupby(resume_df, :race), :call_r = mean(:call))

,race,call_r
,Cat…,Float64
1,black,0.0644764
2,white,0.0965092


## 2.2 Rでデータを部分集合化する

In [38]:
mean(resume_df[resume_df.race .== "black", :call])

0.06447638603696099

In [40]:
resume_bf_df = @chain resume_df begin
    @rsubset(:race == "black", :sex == "female")
    @select(:call, :firstname)
end

,call,firstname
,Int64,Cat…
1,0,Lakisha
2,0,Latonya
3,0,Kenya
4,0,Latonya
5,0,Aisha
6,0,Aisha
7,0,Aisha
8,0,Tamika
9,0,Latonya


In [41]:
resume_bm_df = @chain resume_df begin
    @rsubset(:race == "black", :sex == "male")
    @select(:call, :firstname)
end

resume_wm_df = @chain resume_df begin
    @rsubset(:race == "white", :sex == "male")
    @select(:call, :firstname)
end

resume_wf_df = @chain resume_df begin
    @rsubset(:race == "white", :sex == "female")
    @select(:call, :firstname)
end

,call,firstname
,Int64,Cat…
1,0,Allison
2,0,Kristen
3,0,Carrie
4,0,Jill
5,0,Allison
6,0,Carrie
7,0,Jill
8,0,Allison
9,0,Carrie


In [42]:
mean(resume_wf_df.call) - mean(resume_bf_df.call)

0.0326468944913853

In [43]:
mean(resume_wm_df.call) - mean(resume_bm_df.call)

0.03040785618119901

In [48]:
resume_df = @chain resume_df begin
    @rtransform begin
        :blackfemale = ifelse(:race == "black" && :sex == "female", 1, 0)
    end
end

,firstname,sex,race,call,blackfemale
,Cat…,Cat…,Cat…,Int64,Int64
1,Allison,female,white,0,0
2,Kristen,female,white,0,0
3,Lakisha,female,black,0,1
4,Latonya,female,black,0,1
5,Carrie,female,white,0,0
6,Jay,male,white,0,0
7,Jill,female,white,0,0
8,Kenya,female,black,0,1
9,Latonya,female,black,0,1


In [51]:
println(freqtable(resume_df, :race, :sex, :blackfemale))

2×2×2 Named Array{Int64, 3}

[:, :, blackfemale=0] =
race ╲ sex │ "female"    "male"
───────────┼───────────────────
"black"    │        0       549
"white"    │     1860       575

[:, :, blackfemale=1] =
race ╲ sex │ "female"    "male"
───────────┼───────────────────
"black"    │     1886         0
"white"    │        0         0


In [53]:
levels(resume_df.race)

2-element Vector{String7}:
 "black"
 "white"

In [54]:
levels(resume_df.sex)

2-element Vector{String7}:
 "female"
 "male"

In [55]:
levels(resume_df.firstname)

36-element Vector{String15}:
 "Aisha"
 "Allison"
 "Anne"
 ⋮
 "Tremayne"
 "Tyrone"

## 2.4 ランダム化比較実験

In [5]:
social_df = CSV.read("../../data/CAUSALITY/social.csv", DataFrame, missingstring=["NA"])

,sex,yearofbirth,primary2004,messages,primary2006,hhsize
,String7,Int64,Int64,String15,Int64,Int64
1,male,1941,0,Civic Duty,0,2
2,female,1947,0,Civic Duty,0,2
3,male,1951,0,Hawthorne,1,3
4,female,1950,0,Hawthorne,1,3
5,female,1982,0,Hawthorne,1,3
6,male,1981,0,Control,0,3
7,female,1959,0,Control,1,3
8,male,1956,0,Control,1,3
9,female,1968,0,Control,0,2


In [6]:
describe(social_df)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,sex,,female,,male,0,String7
2,yearofbirth,1956.21,1900,1956.0,1986,0,Int64
3,primary2004,0.401378,0,0.0,1,0,Int64
4,messages,,Civic Duty,,Neighbors,0,String15
5,primary2006,0.312245,0,0.0,1,0,Int64
6,hhsize,2.18442,1,2.0,8,0,Int64


In [18]:
voting_ratio_df = @chain social_df begin
    groupby([:messages])
    @combine(:voting_ratio = mean(:primary2006))
end

control_voting_ratio = @rsubset(voting_ratio_df, :messages == "Control").voting_ratio[1]
@chain voting_ratio_df begin
    @rtransform(:excess_voting_ratio = :voting_ratio - control_voting_ratio)
end

,messages,voting_ratio,excess_voting_ratio
,String15,Float64,Float64
1,Civic Duty,0.314538,0.0178993
2,Hawthorne,0.322375,0.0257363
3,Control,0.296638,0.0
4,Neighbors,0.377948,0.0813099


In [17]:
@rsubset(voting_ratio_df, :messages == "Control").voting_ratio[1]

0.2966383083302395

In [21]:
@chain social_df begin
    groupby([:messages])
    @combine begin
        @astable begin
            :mean_yearofbirth = mean(:yearofbirth)
            :mean_age = 2006 .- :mean_yearofbirth
        end
    end
end

,messages,mean_yearofbirth,mean_age
,String15,Float64,Float64
1,Civic Duty,1956.34,49.659
2,Hawthorne,1956.3,49.7048
3,Control,1956.19,49.8135
4,Neighbors,1956.15,49.8529


In [22]:
@chain social_df begin
    groupby([:messages])
    @combine begin
        @astable begin
            :mean_primary2004 = mean(:primary2004)
            :mean_hhsize = mean(:hhsize)
        end
    end
end

,messages,mean_primary2004,mean_hhsize
,String15,Float64,Float64
1,Civic Duty,0.399445,2.18913
2,Hawthorne,0.40323,2.18014
3,Control,0.400339,2.18367
4,Neighbors,0.406665,2.18777


## 2.5 観察研究

In [23]:
minwage_df = CSV.read("../../data/CAUSALITY/minwage.csv", DataFrame, missingstring=["NA"])

,chain,location,wageBefore,wageAfter,fullBefore,fullAfter,partBefore,partAfter
,String15,String15,Float64,Float64,Float64,Float64,Float64,Float64
1,wendys,PA,5.0,5.25,20.0,0.0,20.0,36.0
2,wendys,PA,5.5,4.75,6.0,28.0,26.0,3.0
3,burgerking,PA,5.0,4.75,50.0,15.0,35.0,18.0
4,burgerking,PA,5.0,5.0,10.0,26.0,17.0,9.0
5,kfc,PA,5.25,5.0,2.0,3.0,8.0,12.0
6,kfc,PA,5.0,5.0,2.0,2.0,10.0,9.0
7,roys,PA,5.0,4.75,2.5,1.0,20.0,25.0
8,burgerking,PA,5.0,5.0,40.0,9.0,30.0,32.0
9,burgerking,PA,5.0,4.5,8.0,7.0,27.0,39.0


In [24]:
describe(minwage_df)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,chain,,burgerking,,wendys,0,String15
2,location,,PA,,southNJ,0,String15
3,wageBefore,4.61771,4.25,4.5,5.75,0,Float64
4,wageAfter,4.99385,4.25,5.05,6.25,0,Float64
5,fullBefore,8.47486,0.0,6.0,60.0,0,Float64
6,fullAfter,8.36173,0.0,6.0,40.0,0,Float64
7,partBefore,18.7542,0.0,16.25,60.0,0,Float64
8,partAfter,18.6885,0.0,17.0,60.0,0,Float64


In [31]:
@chain minwage_df begin
    @rtransform(:state = :location == "PA" ? "PA" : "NJ")
    groupby(:state)
    @combine begin
        :mean_wage_before = mean(:wageBefore)
        :mean_wage_after = mean(:wageAfter)
        :under_threshold_before = mean(:wageBefore .< 5.05)
        :under_threshold_after = mean(:wageAfter .< 5.05)
    end
end

,state,mean_wage_before,mean_wage_after,under_threshold_before,under_threshold_after
,String,Float64,Float64,Float64,Float64
1,PA,4.65134,4.61328,0.940299,0.955224
2,NJ,4.60997,5.08148,0.910653,0.00343643


In [37]:
prop_df = @chain minwage_df begin
    @rtransform begin
        :state = :location == "PA" ? "PA" : "NJ"
        :fulltime_ratio_before = :fullBefore / (:fullBefore + :partBefore)
        :fulltime_ratio_after = :fullAfter / (:fullAfter + :partAfter)
    end
    groupby(:state)
    @combine begin
        :mean_fulltime_ratio_before = mean(:fulltime_ratio_before)
        :mean_fulltime_ratio_after = mean(:fulltime_ratio_after)
    end
    @rtransform :diff_ratio = :mean_fulltime_ratio_after - :mean_fulltime_ratio_before
end

@rsubset(prop_df, :state == "NJ").mean_fulltime_ratio_after - @rsubset(prop_df, :state == "PA").mean_fulltime_ratio_after

1-element Vector{Float64}:
 0.04811886142291416

In [53]:
prop(freqtable(@rtransform(minwage_df, :state = :location == "PA" ? "PA" : "NJ"), :chain, :state), margins=2) |> println

4×2 Named Matrix{Float64}
chain ╲ state │       NJ        PA
──────────────┼───────────────────
"burgerking"  │ 0.405498  0.462687
"kfc"         │ 0.223368  0.149254
"roys"        │ 0.250859  0.223881
"wendys"      │ 0.120275  0.164179


In [54]:
prop_df = @chain minwage_df begin
    @rtransform begin
        :state = :location == "PA" ? "PA" : "NJ"
        :fulltime_ratio_before = :fullBefore / (:fullBefore + :partBefore)
        :fulltime_ratio_after = :fullAfter / (:fullAfter + :partAfter)
    end
    @rsubset(:chain == "burgerking")
    groupby(:state)
    @combine begin
        :mean_fulltime_ratio_before = mean(:fulltime_ratio_before)
        :mean_fulltime_ratio_after = mean(:fulltime_ratio_after)
    end
    @rtransform :diff_ratio = :mean_fulltime_ratio_after - :mean_fulltime_ratio_before
end

@rsubset(prop_df, :state == "NJ").mean_fulltime_ratio_after - @rsubset(prop_df, :state == "PA").mean_fulltime_ratio_after

1-element Vector{Float64}:
 0.03643933939149824

In [55]:
prop_df = @chain minwage_df begin
    @rtransform begin
        :state = :location == "PA" ? "PA" : "NJ"
        :fulltime_ratio_before = :fullBefore / (:fullBefore + :partBefore)
        :fulltime_ratio_after = :fullAfter / (:fullAfter + :partAfter)
    end
    @rsubset(:chain == "burgerking", :location in ["PA", "northNJ", "southNJ"])
    groupby(:state)
    @combine begin
        :mean_fulltime_ratio_before = mean(:fulltime_ratio_before)
        :mean_fulltime_ratio_after = mean(:fulltime_ratio_after)
    end
    @rtransform :diff_ratio = :mean_fulltime_ratio_after - :mean_fulltime_ratio_before
end

@rsubset(prop_df, :state == "NJ").mean_fulltime_ratio_after - @rsubset(prop_df, :state == "PA").mean_fulltime_ratio_after

1-element Vector{Float64}:
 0.031498534750908636

In [56]:
prop_df = @chain minwage_df begin
    @rtransform begin
        :state = :location == "PA" ? "PA" : "NJ"
        :fulltime_ratio_before = :fullBefore / (:fullBefore + :partBefore)
        :fulltime_ratio_after = :fullAfter / (:fullAfter + :partAfter)
    end
    groupby(:state)
    @combine begin
        :mean_fulltime_ratio_before = mean(:fulltime_ratio_before)
        :mean_fulltime_ratio_after = mean(:fulltime_ratio_after)
    end
    @rtransform :diff_ratio = :mean_fulltime_ratio_after - :mean_fulltime_ratio_before
end

@rsubset(prop_df, :state == "NJ").mean_fulltime_ratio_after - @rsubset(prop_df, :state == "NJ").mean_fulltime_ratio_before

1-element Vector{Float64}:
 0.023874740213139733

In [58]:
prop_df = @chain minwage_df begin
    @rtransform begin
        :state = :location == "PA" ? "PA" : "NJ"
        :fulltime_ratio_before = :fullBefore / (:fullBefore + :partBefore)
        :fulltime_ratio_after = :fullAfter / (:fullAfter + :partAfter)
    end
    groupby(:state)
    @combine begin
        :mean_fulltime_ratio_before = mean(:fulltime_ratio_before)
        :mean_fulltime_ratio_after = mean(:fulltime_ratio_after)
    end
    @rtransform :diff_ratio = :mean_fulltime_ratio_after - :mean_fulltime_ratio_before
end

@rsubset(prop_df, :state == "NJ").diff_ratio - @rsubset(prop_df, :state == "PA").diff_ratio

1-element Vector{Float64}:
 0.06155831231224701

## 1変数の記述統計量

In [62]:
prop_df = @chain minwage_df begin
    @rtransform begin
        :state = :location == "PA" ? "PA" : "NJ"
        :fulltime_ratio_before = :fullBefore / (:fullBefore + :partBefore)
        :fulltime_ratio_after = :fullAfter / (:fullAfter + :partAfter)
    end
    groupby(:state)
    @combine begin
        :median_fulltime_ratio_before = median(:fulltime_ratio_before)
        :median_fulltime_ratio_after = median(:fulltime_ratio_after)
    end
    @rtransform :diff_ratio = :median_fulltime_ratio_after - :median_fulltime_ratio_before
end

@rsubset(prop_df, :state == "NJ").median_fulltime_ratio_after - @rsubset(prop_df, :state == "PA").median_fulltime_ratio_after |> println
@rsubset(prop_df, :state == "NJ").median_fulltime_ratio_after - @rsubset(prop_df, :state == "NJ").median_fulltime_ratio_before |> println
@rsubset(prop_df, :state == "NJ").diff_ratio - @rsubset(prop_df, :state == "PA").diff_ratio |> println

[0.07291666666666669]


[0.025000000000000022]
[0.037019230769230804]


In [68]:
prop_df = @chain minwage_df begin
    @rtransform begin
        :state = :location == "PA" ? "PA" : "NJ"
        :fulltime_ratio_before = :fullBefore / (:fullBefore + :partBefore)
        :fulltime_ratio_after = :fullAfter / (:fullAfter + :partAfter)
    end
    @rsubset(:state == "NJ")
end
describe(prop_df, :all)

,variable,mean,std,min,q25,median,q75,max,nunique,nmissing,first,last,eltype
,Symbol,Union…,Union…,Any,Union…,Union…,Union…,Any,Union…,Int64,Any,Any,DataType
1,chain,,,burgerking,,,,wendys,4,0,roys,wendys,String15
2,location,,,centralNJ,,,,southNJ,4,0,centralNJ,northNJ,String15
3,wageBefore,4.60997,0.34349,4.25,4.25,4.5,4.87,5.75,,0,5.0,4.62,Float64
4,wageAfter,5.08148,0.105642,5.0,5.05,5.05,5.05,5.75,,0,5.05,5.14,Float64
5,fullBefore,7.97079,7.98857,0.0,2.25,6.0,11.75,60.0,,0,2.0,0.0,Float64
6,fullAfter,8.4055,7.57518,0.0,2.0,6.0,12.5,40.0,,0,5.0,10.0,Float64
7,partBefore,18.6357,10.4403,0.0,10.0,16.0,25.0,60.0,,0,2.0,33.0,Float64
8,partAfter,18.39,10.7214,0.0,10.5,15.0,25.0,60.0,,0,2.0,24.0,Float64
9,state,,,NJ,,,,NJ,1,0,NJ,NJ,String


In [72]:
iqr(prop_df.wageBefore)

0.6200000000000001

In [73]:
iqr(prop_df.wageAfter)

0.0

In [81]:
quantile(prop_df.wageBefore, range(0, 1, step=0.1))

11-element Vector{Float64}:
 4.25
 4.25
 4.25
 ⋮
 5.0
 5.75

In [82]:
quantile(prop_df.wageAfter, range(0, 1, step=0.1))

11-element Vector{Float64}:
 5.0
 5.05
 5.05
 ⋮
 5.150000000000003
 5.75

In [90]:
sqrt(mean((prop_df.fulltime_ratio_after .- prop_df.fulltime_ratio_before) .^ 2))

0.3014668578470611

In [92]:
mean(prop_df.fulltime_ratio_after .- prop_df.fulltime_ratio_before)

0.02387474021313988

In [93]:
std(prop_df.fulltime_ratio_before)

0.23045922465419544

In [94]:
std(prop_df.fulltime_ratio_after)

0.25100159189283716

In [95]:
var(prop_df.fulltime_ratio_before)

0.053111454228212916

In [96]:
var(prop_df.fulltime_ratio_after)

0.06300179913273839